In [1]:
import numpy as np
from PIL import Image
import os

def destuff_bits(binary_string):
    """
    Removing '1' inserted after every 5 consecutive '0's in the binary string.

    Args:
        binary_string (str): Binary string to be destuffed.

    Returns:
        str: Binary string after destuffing.
    """
    result = ''
    count = 0

    i = 0
    while i < len(binary_string):
        bit = binary_string[i]
        result += bit
        if bit == '0':
            count += 1
            if count == 5:
                # Skip the next bit if it is '1'
                if i + 1 < len(binary_string) and binary_string[i + 1] == '1':
                    i += 1
                count = 0
        else:
            count = 0
        i += 1

    return result

# Constants
PIXEL_COLOR_MAP = {
    (255, 255, 0): '4',  # Yellow
    (255, 0, 0): '3',    # Red
    (0, 255, 0): '2',    # Green
    (255, 255, 255): '1',# White
    (0, 0, 0): '0'       # Black
}
BUS_RATE = 512000  # 512 kbps

def process_image(image_path, initial_timestamp=0):
    image = Image.open(image_path)
    pixels = np.array(image)
    
    rows, cols, _ = pixels.shape
    frames = []
    current_frame = []
    idle_time = 0
    timestamp = initial_timestamp

    in_frame = False
    after_idle = False

    for row in range(rows):
        for col in range(cols):
            pixel = tuple(pixels[row, col])
            if pixel in PIXEL_COLOR_MAP:
                value = PIXEL_COLOR_MAP[pixel]
                if value in '01':
                    if after_idle:
                        frames.append((current_frame, idle_time))
                        current_frame = []
                        idle_time = 0
                        after_idle = False
                    current_frame.append(value)
                    in_frame = True
                elif value == '2':
                    in_frame = False
                    idle_time += 1
                    after_idle = True
                elif value in '34':
                    in_frame = False
                    after_idle = True

    # Append the last frame if there is any
    if current_frame:
        frames.append((current_frame, idle_time))

    dataset = []

    for frame, idle_time in frames:
        binary_string = ''.join(frame)
        print(binary_string)
        binary_string = destuff_bits(binary_string)
        can_id = hex(int(binary_string[1:12], 2))[2:].zfill(3)
        dlc = int(binary_string[16:20], 2)
        if can_id=='130':
            print(binary_string)
            print(dlc)
            print(binary_string[16:20])
            
        data_bytes = [hex(int(binary_string[20 + i*8:28 + i*8], 2))[2:].zfill(2) for i in range(dlc)]
        
        dataset.append({
            'timestamp': round(timestamp, 6),
            'can_id': can_id,
            'dlc': dlc,
            'data': data_bytes
        })
        
        frame_length = len(frame)
        timestamp += (frame_length / BUS_RATE) + (idle_time / BUS_RATE)

    return dataset, timestamp

def save_to_txt(dataset, file_path):
    with open(file_path, 'w') as file:
        for data in dataset:
            data_bytes_str = ','.join(data['data'])
            file.write(f"{data['timestamp']:.6f},{data['can_id']},{data['dlc']},{data_bytes_str}\n")

def process_multiple_images(image_paths, output_file):
    all_data = []
    current_timestamp = 0

    for image_path in image_paths:
        dataset, current_timestamp = process_image(image_path, current_timestamp)
        all_data.extend(dataset)

    save_to_txt(all_data, output_file)



In [3]:
# Example usage
image_folder = "./"
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.png') and f=='original_modify_0.png']
output_file = "original_modify_0.txt"

process_multiple_images(image_paths, output_file)


0010110000010000110000010101000001000001000001000001000001000001000001000001000001000001000001000001000001010011011011111111111
0100111100000100110000010000010000010000010000010010000010000010000010110100111010001000100111010101010111001011111111111
0110100100000100110000010000010000010000010000010000010000011010000010010001000001000001000001000011111000110001011111111111
0000010000011000011000001000001000001000001000001000001000001000001000001000110000010101110101100001111100110101011111111111
0001100011110001100011111110010110100000100000100000100000100000101000001100000100000100000100100110011100111011111111111
0001001100000100110000010111011000001000001000001111111111111111001111111000001101001100110001000010111111011111111111
0001001100000001100000011101100000000000000011111111111111100111111100000101001100110001000010111111011111111111
8
1000
0001001100010001100000100010110000010000010000010000010000101111001111111000001101000011100110001101100101011111111111
00010100000100001100